I'm going to compare the NNs vs Peak Counts my making a peak counts emulator. This is where I'll try loading them up and making the training data and seeing what they look like. 

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import numpy as np 
import h5py
from lenstools import ConvergenceMap
from scipy.ndimage import gaussian_filter
from astropy.units import deg

In [ ]:
training_filename = '/home/users/swmclau2/oak/Uatu/UatuFastPMTraining/UatuFastPMTraining.hdf5'
test_filename = '/home/users/swmclau2/oak/Uatu/UatuFastPMTest/UatuFastPMTest.hdf5'

In [ ]:
with h5py.File(training_filename, 'r') as f:
    mean, std = f.attrs['mean'], f.attrs['std']
    X =  f['Box000']['X'][()].squeeze()

In [1]:
plt.imshow(np.log10(X[0]-X[0].min()+1e-6))

NameError: name 'plt' is not defined

In [ ]:
shape_noise = 0.3/np.sqrt((2.34**2)*30)

In [ ]:
def noise_and_smooth(image, noise_level = shape_noise, smooth=1):
    i = image + np.random.randn(*image.shape)*noise_level
    return gaussian_filter(i, smooth)

In [ ]:
smoothX = noise_and_smooth(X[0])
plt.imshow(smoothX)

In [ ]:
smoothX.min()

In [ ]:
plt.hist(smoothX.flatten())

In [ ]:
def image_pc(image):
    #image = (image-image.mean())/image.std()
    cmap = ConvergenceMap(image, angle=10*deg)
    thresholds = np.linspace(-0.01, 0.04,  41)
    #thresholds = np.linspace(-5, 5, 200)
    nu,peaks = cmap.peakCount(thresholds, norm=False)
    return nu,peaks#/psd1D[0]

In [ ]:
nu,pc = image_pc(smoothX)

In [ ]:
plt.plot(nu, pc)
plt.yscale('log')

In [ ]:
def compute_all_pc(images):
    
    all_pcs = np.zeros((images.shape[0], 40))
    for i, im in enumerate(images):
        nu, all_pcs[i] = image_pc(noise_and_smooth(im))
    return nu, all_pcs

In [ ]:
nu, all_pcs = compute_all_pc(X)

In [ ]:
plt.errorbar(nu, all_pcs.mean(axis=0), yerr = all_pcs.std(axis=0))
plt.yscale('log')

In [ ]:
# create the peak counts datasets

def create_peak_counts_dset(conv_dset_fname):
        
    with h5py.File(conv_dset_fname, 'r') as cf:
        
        mean_pc = np.zeros((len(cf.keys()), 40))
        #err_pc =  np.zeros((len(cf.keys()), 40))
        cov_pc = np.zeros((len(cf.keys()), 40, 40))
        for i,key in enumerate(cf.keys()):
            print key
            X =  cf[key]['X'][()].squeeze()
            Y =  cf[key]['Y'][()].squeeze()
            
            _, all_pcs = compute_all_pc(X)
            
            mean_pc[i] = all_pcs.mean(axis=0)
            #err_pc[i] = all_pcs.std(axis=0)
            cov_pc[i] = all_pcs.cov(axis=0)
            
        return mean_pc, cov_pc

In [ ]:
test_pc, test_err = create_peak_counts_dset(test_filename)

In [ ]:
np.save('test_pc.npy', test_pc)
np.save('test_err.npy', test_err)

In [ ]:
training_pc.shape